In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from pylab import rcParams
from IPython.display import display
import math

import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
# pd.set_option('future.no_silent_downcasting', False)


#### Data preparation

In [3]:
# open and read excel file
df = pd.read_excel("C:/Users/Acer/Desktop/Madi/Tasks/Nowcasting/Data/input data/Данные-рус Обработанные.xlsx")
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
3,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
4,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88


In [4]:
# remove first two rows (blank)
df = df.drop(index=df.index[:2]).reset_index(drop=True)
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
2,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
3,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
4,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66


In [5]:
# Set the first row as column labels
df.columns = df.iloc[0]  # Assign first row as header
df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [6]:
# convert date columns to datetime format
df.columns = list(df.columns[:3]) + list(pd.to_datetime(df.columns[-120:]))
df = df.rename(columns={col: col.strftime("%Y-%m") for col in df.columns[-120:]}) # change "2015-01-01 00:00:00" to "2015-01" 
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [7]:
# check values
print(df["Показатель"].unique())
print(df["Категория хозяйства"].unique())
print(df["Регион"].unique())

['КРС' 'Овцы и козы' 'Свиньи' 'Лошади' 'Верблюды' 'Птицы' 'Молоко' 'Яйца']
['Сельхозпредприятия' 'Крестьянские или фермерские хозяйства'
 'Хозяйства населения']
['РЕСПУБЛИКА КАЗАХСТАН' '    АКМОЛИНСКАЯ ОБЛАСТЬ'
 '    АКТЮБИНСКАЯ ОБЛАСТЬ' '    АЛМАТИНСКАЯ ОБЛАСТЬ'
 '    АТЫРАУСКАЯ ОБЛАСТЬ' '    ЗАПАДНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ЖАМБЫЛСКАЯ ОБЛАСТЬ' '    КАРАГАНДИНСКАЯ ОБЛАСТЬ'
 '    КОСТАНАЙСКАЯ ОБЛАСТЬ' '    КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ'
 '    МАНГИСТАУСКАЯ ОБЛАСТЬ' '    ЮЖНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ПАВЛОДАРСКАЯ ОБЛАСТЬ' '    СЕВЕРО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ВОСТОЧНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ' '    ГАСТАНА' '    ГАЛМАТЫ'
 '    ГШЫМКЕНТ' '    ТУРКЕСТАНСКАЯ ОБЛАСТЬ' '    ОБЛАСТЬ ЖЕТІСУ'
 '    ОБЛАСТЬ ҰЛЫТАУ' '    ОБЛАСТЬ АБАЙ']


In [8]:
df["Регион"] = df["Регион"].str.strip() # strip unnecessary spaces

rename_dict = {
    "ВОСТОЧНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ", 
    "ГАЛМАТЫ": "ГОРОД АЛМАТЫ",
    "ГАСТАНА": "ГОРОД АСТАНА",
    "ГШЫМКЕНТ": "ГОРОД ШЫМКЕНТ",  
    "ЗАПАДНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",
    "СЕВЕРО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",
    "ЮЖНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",  
}

# Rename region names 
df["Регион"] = df["Регион"].replace(rename_dict)

df["Регион"].unique()

array(['РЕСПУБЛИКА КАЗАХСТАН', 'АКМОЛИНСКАЯ ОБЛАСТЬ',
       'АКТЮБИНСКАЯ ОБЛАСТЬ', 'АЛМАТИНСКАЯ ОБЛАСТЬ', 'АТЫРАУСКАЯ ОБЛАСТЬ',
       'ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ЖАМБЫЛСКАЯ ОБЛАСТЬ',
       'КАРАГАНДИНСКАЯ ОБЛАСТЬ', 'КОСТАНАЙСКАЯ ОБЛАСТЬ',
       'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ', 'МАНГИСТАУСКАЯ ОБЛАСТЬ',
       'ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ПАВЛОДАРСКАЯ ОБЛАСТЬ',
       'СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ',
       'ГОРОД АСТАНА', 'ГОРОД АЛМАТЫ', 'ГОРОД ШЫМКЕНТ',
       'ТУРКЕСТАНСКАЯ ОБЛАСТЬ', 'ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ',
       'ОБЛАСТЬ АБАЙ'], dtype=object)

In [9]:
# remove ЮКО
df = df[df["Регион"] != "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"]
df.sample(10)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
520,Яйца,Хозяйства населения,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,4250.1,5193.1,8476.2,9801.2,16476.1,13275.2,13275.2,...,3889.2,4720.3,6221.4,7206.2,5819.4,5475.6,5286.5,4241.7,3976.5,4110.7
147,Свиньи,Сельхозпредприятия,ГОРОД АСТАНА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,Яйца,Крестьянские или фермерские хозяйства,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,4.4,4.5,9.2,21.9,12.5,56.7,56.7,...,1.7,10.1,11,19.4,19.1,19.9,5.7,2.9,3.3,3.3
329,Верблюды,Хозяйства населения,ОБЛАСТЬ АБАЙ,нет,нет,нет,нет,нет,нет,нет,...,0,0,0,0,0,0,0,0,0,14.4
146,Свиньи,Сельхозпредприятия,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,24.09,29.67,29.79,35.83,35.98,52.76,35.4,...,151.08,168.74,161.3,139.41,162.44,136.47,138.19,142.93,121.5,145.41
241,Лошади,Крестьянские или фермерские хозяйства,ОБЛАСТЬ АБАЙ,нет,нет,нет,нет,нет,нет,нет,...,1856.85,1755.2,2180.3,4554.7,2585.48,4382.68,5216.5,2388.7,1526.3,4072.3
372,Птицы,Крестьянские или фермерские хозяйства,ОБЛАСТЬ ҰЛЫТАУ,нет,нет,нет,нет,нет,нет,нет,...,NaN,NaN,NaN,NaN,NaN,0.1,0.2,0.2,0.7,0.3
274,Верблюды,Сельхозпредприятия,МАНГИСТАУСКАЯ ОБЛАСТЬ,8.16,2.26,6.54,3.71,5.3,0.43,0.37,...,0,244.3,NaN,NaN,NaN,251,250,NaN,NaN,500
381,Птицы,Хозяйства населения,КАРАГАНДИНСКАЯ ОБЛАСТЬ,3.41,9.85,49.15,38.89,12.25,99.56,36.29,...,17.4,12.2,8.3,45.7,48.1,9.5,52.9,51,30.1,90.6
85,Овцы и козы,Сельхозпредприятия,ОБЛАСТЬ ЖЕТІСУ,нет,нет,нет,нет,нет,нет,нет,...,103.98,24.8,5.63,21.46,6.37,12.13,9.16,6.77,79.05,2.97


In [10]:
# fill missing values with 0
df = df.fillna(0)

# replace "нет" with NA (Абай, Жетісу, Ұлытау, г.Шымкент, Туркестанская до их образования)
df.replace("нет", np.nan, inplace=True)
df.sample(10)

C:\Users\Acer\AppData\Local\Temp\ipykernel_10412\335651453.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\Acer\AppData\Local\Temp\ipykernel_10412\335651453.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("нет", np.nan, inplace=True)


,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
56,КРС,Хозяйства населения,ПАВЛОДАРСКАЯ ОБЛАСТЬ,1713.84,1842.35,1924.07,1522.17,1613.78,1819.12,2167.86,...,1371.10,1076.00,1249.00,1415.90,1470.00,1043.30,1210.30,1802.60,2986.10,3348.60
5,КРС,Сельхозпредприятия,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,36.38,16.14,148.50,145.26,131.27,115.01,60.60,...,333.47,438.04,378.20,585.85,607.34,944.69,517.31,622.35,478.05,964.38
9,КРС,Сельхозпредприятия,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,9.70,15.30,21.82,13.13,6.71,8.39,10.09,...,51.07,118.23,104.68,137.22,43.34,71.07,55.35,79.58,41.26,110.81
225,Лошади,Крестьянские или фермерские хозяйства,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,356.84,236.90,511.54,325.77,317.45,553.81,340.77,...,524.80,573.20,562.50,813.70,252.50,222.70,789.10,379.70,753.20,1232.20
373,Птицы,Крестьянские или фермерские хозяйства,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.90,2.40,0.00,0.20,2.30,4.90,0.00,1.30,4.60,1.40
384,Птицы,Хозяйства населения,МАНГИСТАУСКАЯ ОБЛАСТЬ,0.00,0.00,0.00,0.00,0.00,0.51,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
127,Овцы и козы,Хозяйства населения,ГОРОД ШЫМКЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.60,43.70,46.40,44.20,42.00,39.70,50.70,49.00,46.20,174.80
422,Молоко,Крестьянские или фермерские хозяйства,АТЫРАУСКАЯ ОБЛАСТЬ,253.70,232.30,317.00,503.10,1045.90,674.90,674.90,...,211.10,320.10,687.30,681.60,669.00,619.70,457.80,352.90,633.20,680.10
111,Овцы и козы,Хозяйства населения,АКМОЛИНСКАЯ ОБЛАСТЬ,537.76,575.22,614.47,609.03,783.18,792.67,519.91,...,647.70,602.10,664.20,711.70,394.90,429.20,564.70,580.90,710.10,551.00
410,Молоко,Сельхозпредприятия,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1462.70,1418.10,1694.40,1735.50,2147.90,2062.90,2062.90,...,2674.60,2532.10,2611.20,2509.30,2476.30,2470.80,2468.20,2469.90,2428.00,2547.70


In [11]:
# check replacement
df.groupby("Регион")[df.columns[3:]].apply(lambda g: g.isna().sum()).reset_index()

,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,АКТЮБИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,АЛМАТИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,АТЫРАУСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,ГОРОД АЛМАТЫ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ГОРОД АСТАНА,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ГОРОД ШЫМКЕНТ,24,24,24,24,24,24,24,24,24,...,0,0,0,0,0,0,0,0,0,0
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 

In [12]:
# get the final data by summing across показатели & категория хозяйства
totals = df.groupby(["Показатель", "Регион"])[df.columns[3:]].agg(lambda x: x.sum() if x.notna().any() else np.nan).reset_index()
totals


,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,...,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,...,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,...,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,...,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70,4.20,1.80,...,0.0,0.8,0.00,0.0,0.50,0.0,0.00,0.40,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Яйца,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1569.2,1156.4,1174.80,1957.4,1310.40,973.5,1757.90,961.60,806.3,1780.60
164,Яйца,ПАВЛОДАРСКАЯ ОБЛАСТЬ,9637.30,8361.40,9164.50,10447.60,14083.60,14200.20,14200.20,12827.60,...,16631.1,16747.1,17414.00,18832.4,18654.70,18689.8,18071.90,16535.50,15899.2,14834.90
165,Яйца,РЕСПУБЛИКА КАЗАХСТАН,325795.20,303234.00,376664.50,400243.30,462679.40,454045.70,454045.70,431318.00,...,381480.5,368147.0,377486.90,393946.9,397493.70,394871.5,385885.00,371208.00,362418.8,364206.50
166,Яйца,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,37704.50,37570.60,41732.40,52925.70,68018.40,79389.60,79389.60,57759.90,...,49356.3,49099.2,51500.30,56233.7,58261.10,53762.5,51783.90,47959.40,47352.0,48441.50


### Concat with other indicators dataset (температура, осадки, поголовье)

In [14]:
x_vars = pd.read_excel("C:/Users/Acer/Desktop/Madi/Tasks/Nowcasting/Data/Data cleansing/output data/доп регрессоры обработанные.xlsx")
x_vars.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Температура,АКМОЛИНСКАЯ ОБЛАСТЬ,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,...,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
1,Температура,АКТЮБИНСКАЯ ОБЛАСТЬ,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,...,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
2,Температура,АЛМАТИНСКАЯ ОБЛАСТЬ,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,...,2.812903,11.100000,16.348387,22.023333,23.158065,23.487097,13.726667,9.222581,2.256667,-6.796774
3,Температура,АТЫРАУСКАЯ ОБЛАСТЬ,-6.222581,-6.300000,0.380645,10.856667,19.506452,28.906667,27.177419,25.241935,...,2.522581,17.776667,17.341935,27.453333,28.264516,25.774194,19.956667,10.648387,3.096667,-2.612903
4,Температура,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,-11.845161,-8.928571,-4.406452,7.470000,17.100000,25.283333,22.690323,20.348387,...,-4.632258,14.926667,13.332258,23.853333,24.132258,21.151613,15.246667,8.261290,0.423333,-5.496774


In [15]:
df_combined = pd.concat([totals, x_vars], ignore_index=True)
df_combined.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
120,Птицы,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.400000e+00,175.40,2.354000e+02,2.540000e+02,1.171000e+02,1.380000e+01,9.80,1.407000e+02,58.10,5.720000e+01
178,Температура,ПАВЛОДАРСКАЯ ОБЛАСТЬ,-1.382258e+01,-1.250714e+01,-6.219355e+00,7.01,1.562258e+01,21.33,2.148710e+01,1.852258e+01,...,-3.677419e+00,8.05,1.204516e+01,2.175667e+01,2.243226e+01,1.919355e+01,10.70,4.567742e+00,-4.35,-1.125806e+01
45,Лошади,АТЫРАУСКАЯ ОБЛАСТЬ,4.541600e+02,5.202400e+02,5.060300e+02,400.49,7.934200e+02,472.80,3.068100e+02,3.807400e+02,...,8.877000e+02,548.90,9.242000e+02,8.789000e+02,4.853000e+02,6.833000e+02,1180.00,1.064900e+03,1411.84,2.715720e+03
163,Яйца,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.569200e+03,1156.40,1.174800e+03,1.957400e+03,1.310400e+03,9.735000e+02,1757.90,9.616000e+02,806.30,1.780600e+03
33,КРС,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,1.446340e+03,1.165320e+03,1.233920e+03,1355.81,1.189730e+03,1393.93,1.571370e+03,1.598740e+03,...,1.089570e+03,1336.93,1.269080e+03,1.472820e+03,1.271540e+03,1.581370e+03,1724.95,1.919480e+03,1838.46,2.285110e+03
272,Поголовье: лошади,АКТЮБИНСКАЯ ОБЛАСТЬ,7.733500e+04,8.354200e+04,8.585700e+04,93332.00,1.021030e+05,101727.00,1.021400e+05,1.007710e+05,...,2.798580e+05,285041.00,3.046280e+05,3.176360e+05,3.174390e+05,3.290970e+05,328082.00,3.229610e+05,320196.00,3.112440e+05
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,1.019800e+02,6.747000e+01,3.748400e+02,115.59,2.187200e+02,14.15,1.977000e+01,3.000000e+00,...,3.910000e+02,126.10,2.291000e+02,1.220000e+01,2.410000e+01,5.700000e+00,49.87,6.300000e+01,154.30,5.384000e+02
61,Лошади,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,7.292400e+02,2.725400e+02,1.895900e+02,279.55,5.002600e+02,1330.63,1.892810e+03,2.135990e+03,...,4.774900e+02,472.46,5.007700e+02,1.445380e+03,1.960900e+03,2.428740e+03,2033.61,8.338800e+02,1068.47,1.523630e+03
237,Осадки,КОСТАНАЙСКАЯ ОБЛАСТЬ,2.280000e+01,3.400000e+00,1.490000e+01,20.10,8.230000e+01,37.60,4.790000e+01,2.300000e+01,...,1.410000e+01,31.40,3.020000e+01,5.180000e+01,5.600000e+01,7.130000e+01,10.50,8.900000e+00,37.00,6.500000e+00
291,Поголовье: лошади,РЕСПУБЛИКА КАЗАХСТАН,1.819378e+06,1.886719e+06,1.976977e+06,2177690.00,2.308349e+06,2310386.00,2.228859e+06,2.184637e+06,...,3.827076e+06,3985124.00,4.267566e+06,4.483555e+06,4.488493e+06,4.313452e+06,4274920.00,4.193227e+06,4132104.00,4.072288e+06


In [16]:
# save into an excel file: все 8 показателей в сумме по категориям хозяйств в одном excel
df_combined.to_excel("output data/Просуммированные по категориям с доп регрессорами.xlsx", index=False)

In [ ]:
# # save into excel files: 8 показателей в сумме по категориям хозяйств, каждый в отдельном excel

# indicators = ["КРС", "Лошади", "Свиньи", "Овцы и козы", "Птицы", "Молоко", "Яйца", "Верблюды"]

# for indicator in indicators:
#     df_filtered = df_combined[df_combined["Показатель"] == indicator].drop(columns=["Показатель"])
#     file_name = f"output data/{indicator}_обработанные.xlsx"
#     df_filtered.to_excel(file_name, index=False)
